In [1]:
from Models import *
from Utils import *

Using TensorFlow backend.


In [4]:
LETTER_GRAM_SIZE = 3 # See section 3.2.
WINDOW_SIZE = 3 # See section 3.2.
TOTAL_LETTER_GRAMS = 5005
WORD_DEPTH = WINDOW_SIZE * TOTAL_LETTER_GRAMS # See equation (1).
K = 300 # Dimensionality of the max-pooling layer. See section 3.4.
L = 128 # Dimensionality of latent semantic space. See section 3.5.
J = 1 # Number of random unclicked documents serving as negative examples for a query. See section 4.
FILTER_LENGTH = 1 # We only consider one time step for convolutions.

nb_words = 50005
max_len = 10
batch_size = 128


# load pre-trained trigram
tokeniser = L3wTransformer(TOTAL_LETTER_GRAMS)
tokeniser = tokeniser.load("/work/data/trigram/2M_50k_trigram")

In [6]:
df_may, qrel_may = get_test_data("MayFlower")
df_june, qrel_june = get_test_data("JuneFlower")

b'Skipping line 19898: expected 6 fields, saw 8\nSkipping line 20620: expected 6 fields, saw 8\nSkipping line 38039: expected 6 fields, saw 8\n'


In [8]:


q_may = to_categorical(parse_texts(df_may.q.tolist(), tokeniser, max_len), nb_words)
d_may = to_categorical(parse_texts(df_may.d.tolist(), tokeniser, max_len), nb_words)

q_may = q_may.reshape(len(df_may), max_len, nb_words)
d_may = d_may.reshape(len(df_may), max_len, nb_words)

q_june = to_categorical(parse_texts(df_june.q.tolist(), tokeniser, max_len), nb_words)
d_june = to_categorical(parse_texts(df_june.d.tolist(), tokeniser, max_len), nb_words)

q_june = q_june.reshape(len(df_june), max_len, nb_words)
d_june = d_june.reshape(len(df_june), max_len, nb_words)

In [5]:
file_dir = '/data/t-mipha/data/agi_encoder/v4/universal/CLICKED_QQ_EN_universal_train_1M.txt'

reader = pd.read_csv(file_dir, chunksize=batch_size, iterator=True, usecols=[0,1], names=["q", "d"], sep="\t", header=None, error_bad_lines=False)

def qq_batch_generator(reader, tokeniser, batch_size, max_len, nb_words):
    for df in reader:
        q = df.q.tolist()
        d = [i.split("<sep>")[0] for i in df.d.tolist()]
        
        q = pad_sequences(tokeniser.texts_to_sequences(q), maxlen=max_len)
        d = pad_sequences(tokeniser.texts_to_sequences(d), maxlen=max_len)
        
        
        q_one_hot = to_categorical(q, nb_words)   
        q_one_hot = q_one_hot.reshape(batch_size, max_len, nb_words)
        
        d_one_hot = to_categorical(d, nb_words)   
        d_one_hot = q_one_hot.reshape(batch_size, max_len, nb_words)
            
            
        # negative sampling from positive pool
        neg_d_one_hot = [[] for j in range(J)]
        for i in range(batch_size):
            possibilities = list(range(batch_size))
            possibilities.remove(i)
            negatives = np.random.choice(possibilities, J, replace = False)
            for j in range(J):
                negative = negatives[j]
                neg_d_one_hot[j].append(d_one_hot[negative].tolist())
        
        y = np.zeros((batch_size, J + 1))
        y[:, 0] = 1
        
        for j in range(J):
            neg_d_one_hot[j] = np.array(neg_d_one_hot[j])
        
        
        yield [q_one_hot, d_one_hot] + [neg_d_one_hot[j] for j in range(J)], y

        
clsm.model.fit_generator(qq_batch_generator(reader, tokeniser, batch_size, max_len, nb_words), steps_per_epoch=2, epochs=1, verbose=2, callbacks=[TQDMNotebookCallback()])       
cosine = CosineSim(L)


# for q, d, qrel, df in [[q_may, d_may, qrel_may, df_may], [q_june, d_june, qrel_june, df_june]]:
#     pred = cosine.model.predict([clsm.encoder.predict(q), clsm.encoder.predict(d)])
#     pred = convert_2_trec(df.q.tolist(), df.d.tolist(), pred, False)
#     evaluate(qrel, pred)       

Epoch 1/1
 - 53s - loss: 0.6924



/work/workspace/Models.py:26: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  pred = merge([q_input, d_input], mode="cos")
/home/t-jamano/.local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [84]:
batch_size = 10
file_dir = '/data/t-mipha/data/query_similarity_ndcg/JuneFlowerIdeal.txt'
reader = pd.read_csv(file_dir, chunksize=batch_size, iterator=True,names=["market", "qid", "q", "label", "d", "date"], sep="\t", header=None, error_bad_lines=False)

def encoder_batch_generator(reader, tokeniser, batch_size, max_len, nb_words, mode="q"):
    
    while True:
        for df in reader:
            
            q = df.q.tolist() if mode == "q" else df.d.tolist()

            q = pad_sequences(tokeniser.texts_to_sequences(q), maxlen=max_len)

            q_one_hot = to_categorical(q, nb_words)   
            q_one_hot = q_one_hot.reshape(batch_size, max_len, nb_words)

            yield q_one_hot


In [85]:
file_dir = '/data/t-mipha/data/query_similarity_ndcg/JuneFlowerIdeal.txt'
df = pd.read_csv(file_dir,names=["market", "qid", "q", "label", "d", "date"], sep="\t", header=None, error_bad_lines=False)


In [86]:
batch_size

10

In [87]:
%%time
clsm.encoder.predict_generator(encoder_batch_generator(reader, tokeniser, batch_size, max_len, nb_words, "q"), steps=10000).shape

Exception in thread Thread-33:
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/t-jamano/.local/lib/python3.6/site-packages/keras/utils/data_utils.py", line 579, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-84-eb054b4f5aa5>", line 15, in encoder_batch_generator
    q_one_hot = q_one_hot.reshape(batch_size, max_len, nb_words)
ValueError: cannot reshape array of size 3500350 into shape (10,10,50005)



StopIteration: 

In [35]:
%%time
q_may = to_categorical(parse_texts(df_may.q.tolist()[:100], tokeniser, max_len), nb_words)
q_may = q_may.reshape(100, max_len, nb_words)

CPU times: user 0 ns, sys: 9.44 s, total: 9.44 s
Wall time: 9.11 s


TypeError: predict_generator() missing 1 required positional argument: 'steps'

In [16]:
clsm.encoder.predict(q_may).shape

(1000, 128)